In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
import time

In [83]:
def raw_to_cpm(df):
    column_sums = df.sum(axis=0)
    cpm_df = (df / column_sums) * 1e6
    return cpm_df

In [84]:
sig = pd.read_csv('signature_matrix_pbmc.csv', index_col=0)
sig = sig[~np.all(sig == 0, axis=1)].sort_index()
sig = stats.zscore(sig, axis=0, ddof=1)
sig

,Monocyte,T_cells,NK_cell,B_cell,Pre-B_cell_CD34-,HSC_-G-CSF,GMP,Platelets,CMP,MEP,Pro-B_cell_CD34+,Neutrophils
A1BG,1.197126,-0.648752,1.685982,-0.648666,-0.648614,1.817633,-0.648223,-0.646906,-0.647678,-0.647173,-0.647206,-0.647030
A2M-AS1,-0.646642,-0.648752,1.068951,-0.648666,-0.648614,0.989672,0.790101,-0.646906,-0.647678,1.093615,-0.647206,-0.647030
AAGAB,-0.646642,-0.648752,-0.649374,-0.648666,1.967520,-0.648836,-0.648223,-0.646906,1.248565,-0.647173,-0.647206,-0.647030
AAK1,-0.646642,-0.648752,-0.649374,-0.648666,-0.648614,-0.648836,-0.648223,-0.646906,1.793630,-0.647173,1.609803,1.040067
AAMP,-0.646642,-0.648752,-0.649374,-0.648666,-0.648614,-0.648836,-0.648223,-0.646906,-0.647678,1.081292,1.653081,-0.647030
...,...,...,...,...,...,...,...,...,...,...,...,...
ZWILCH,-0.646642,1.615555,-0.649374,-0.648666,-0.648614,-0.648836,-0.648223,-0.646906,-0.647678,-0.647173,1.964691,-0.647030
ZWINT,-0.646642,-0.648752,1.397472,-0.648666,-0.648614,-0.648836,1.449269,-0.646906,-0.647678,-0.647173,-0.647206,-0.647030
ZYG11B,1.232254,-0.648752,-0.649374,-0.648666,-0.648614,-0.648836,-0.648223,1.258626,-0.647678,-0.647173,-0.647206,-0.647030
ZYX,-0.646642,1.338755,1.313106,-0.648666,1.360018,1.148175,-0.648223,-0.646906,-0.647678,-0.647173,-0.647206,-0.647030


In [85]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [86]:
data = pd.read_csv('PBMC_PSEUDOBULK.csv', index_col=(0))
data = data[data.index.isin(sig.index)].sort_index()
data = raw_to_cpm(data)
data = np.log2(data + 1).T
data = scaler.fit_transform(data)
#data = stats.zscore(data, axis=0, ddof=1).T
data

array([[-0.6686403 ,  0.24522794, -3.24018172, ..., -0.86296058,
        -1.66683817, -0.54221212],
       [-0.86106881,  0.74482723,  0.37196957, ..., -0.44384575,
        -1.25391813, -0.6520409 ],
       [-1.84409071,  1.07379469, -1.47414958, ...,  0.01165742,
        -0.94284693,  0.09897342],
       ...,
       [-0.63206914, -0.36436069, -0.17611065, ...,  0.1669943 ,
        -2.17964703, -0.49825435],
       [ 0.26714879,  0.35252478, -0.3162183 , ...,  1.08568629,
        -0.67301063,  0.095981  ],
       [ 0.93267242,  2.23222671,  1.9520168 , ...,  0.41317095,
         1.52971975, -1.23460624]])

In [87]:
meta = pd.read_csv('cell_fractions_PBMC.csv', index_col=0)
meta

,Monocyte,T_cells,NK_cell,B_cell,Pre-B_cell_CD34-,HSC_-G-CSF,GMP,Platelets,CMP,MEP,Pro-B_cell_CD34+,Neutrophils
random_sample1,0.006277,0.165013,0.075116,0.094756,0.069447,0.087265,0.119052,0.105082,0.141729,0.117028,0.012553,0.006682
random_sample2,0.054440,0.090691,0.110522,0.063282,0.076670,0.086523,0.046861,0.071239,0.113174,0.121511,0.064545,0.100543
random_sample3,0.062670,0.088145,0.105129,0.057065,0.060462,0.104280,0.008662,0.126868,0.126868,0.147079,0.110054,0.002717
random_sample4,0.100299,0.117598,0.000832,0.103959,0.133400,0.136727,0.024784,0.007984,0.108782,0.037425,0.142881,0.085329
random_sample5,0.147315,0.001800,0.041704,0.030003,0.123612,0.043204,0.127813,0.129013,0.087459,0.098260,0.103510,0.066307
...,...,...,...,...,...,...,...,...,...,...,...,...
random_sample996,0.076587,0.125000,0.005343,0.133258,0.034326,0.085330,0.084035,0.095207,0.130991,0.041937,0.117552,0.070434
random_sample997,0.073278,0.116649,0.094770,0.086872,0.089979,0.100207,0.076644,0.103055,0.000388,0.099301,0.066287,0.092569
random_sample998,0.100911,0.029690,0.061749,0.142077,0.029508,0.099454,0.080328,0.036430,0.170856,0.169763,0.059745,0.019490
random_sample999,0.134146,0.136950,0.078497,0.035043,0.082282,0.047799,0.074572,0.079058,0.091113,0.116204,0.056350,0.067984


In [88]:
data_test = pd.read_csv('PBMC_PSEUDOBULK_INDEPENDENT.csv', index_col=(0))
data_test = data_test[data_test.index.isin(sig.index)].sort_index()
data_test = raw_to_cpm(data_test)
data_test = np.log2(data_test + 1).T
data_test = scaler.transform(data_test)
data_test

array([[ 0.5339842 , -0.06552027, -1.59144093, ..., -1.9409341 ,
         0.98370543,  1.10165987],
       [ 0.65779575,  0.33443127, -0.41409247, ..., -0.87350529,
        -0.6803911 , -0.50299834],
       [ 0.25904538,  0.79357057,  0.2212664 , ...,  0.4937696 ,
         1.31880031, -0.12369224],
       ...,
       [-0.16070252,  0.50607147, -0.27656719, ...,  1.0119844 ,
         0.16508726,  0.49683043],
       [ 0.28797965, -0.89304183,  0.67650769, ..., -0.20495834,
        -0.14154555, -1.2733377 ],
       [-0.55605203, -0.33649687,  0.61737798, ...,  0.75366539,
        -0.66470001,  0.01743011]])

In [89]:
meta_test = pd.read_csv('cell_fractions_PBMC_INDEPENDENT.csv', index_col=0)
meta_test

,Monocyte,T_cells,NK_cell,B_cell,Pre-B_cell_CD34-,HSC_-G-CSF,GMP,Platelets,CMP,MEP,Pro-B_cell_CD34+,Neutrophils
random_sample1,0.038309,0.111131,0.018251,0.117456,0.086014,0.088001,0.094145,0.131189,0.025298,0.054572,0.163715,0.071919
random_sample2,0.032438,0.019936,0.023315,0.038351,0.166751,0.151546,0.067748,0.111843,0.077716,0.100693,0.106437,0.103227
random_sample3,0.041416,0.019781,0.041570,0.061351,0.025344,0.129810,0.151290,0.086694,0.136455,0.022253,0.145727,0.138309
random_sample4,0.101241,0.085955,0.188968,0.056491,0.009083,0.004209,0.132477,0.005760,0.105671,0.120292,0.035667,0.154187
random_sample5,0.097716,0.089233,0.092985,0.097716,0.004731,0.153344,0.030343,0.108320,0.100979,0.116639,0.077977,0.030016
...,...,...,...,...,...,...,...,...,...,...,...,...
random_sample496,0.148879,0.056143,0.096323,0.038565,0.047534,0.119641,0.038744,0.106726,0.144753,0.053094,0.116592,0.033004
random_sample497,0.100766,0.039655,0.017433,0.078544,0.157088,0.001916,0.141954,0.094444,0.115900,0.068582,0.073946,0.109770
random_sample498,0.131689,0.077225,0.064490,0.091587,0.092399,0.096328,0.119361,0.054329,0.070180,0.066793,0.081425,0.054193
random_sample499,0.028780,0.003111,0.096142,0.060828,0.139235,0.022246,0.140012,0.091942,0.106565,0.132234,0.057094,0.121811


In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [91]:
from sklearn.model_selection import train_test_split
import torch

# Split the data into 80% training and 20% validation sets
X_train, X_val, Y_train, Y_val = train_test_split(data, meta.to_numpy(), test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).to(device)

signature_matrix_pbmc_expanded_tensor = torch.tensor(sig.values, dtype=torch.float32).to(device)

X_test_tensor = torch.tensor(data_test, dtype=torch.float32).to(device)
Y_test_tensor = torch.tensor(meta_test.to_numpy(), dtype=torch.float32).to(device)

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define DeconvolutionModel1 with all components
class DeconvolutionModel1(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim, output_dim, signature_matrix):
        super(DeconvolutionModel1, self).__init__()
        self.encoder = self.Encoder(input_dim, hidden_dim, attention_dim)
        self.decoder = self.Decoder(hidden_dim, output_dim, signature_matrix)
        self.optimizer = optim.Adam(self.parameters(), lr=1e-4)  # Optimizer
        self.signature_matrix = signature_matrix  # Store the signature matrix

    class AttentionBlock(nn.Module):
        def __init__(self, input_dim, hidden_dim):
            super(DeconvolutionModel1.AttentionBlock, self).__init__()
            self.attn = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8, dropout=0.1)
            self.fc = nn.Linear(input_dim, hidden_dim)

        def forward(self, x):
            attn_output, _ = self.attn(x, x, x)  # Self-attention
            x = self.fc(attn_output)  # Linear transformation
            return x

    class Encoder(nn.Module):
        def __init__(self, input_dim, hidden_dim, attention_dim):
            super(DeconvolutionModel1.Encoder, self).__init__()
            self.fc1 = nn.Linear(input_dim, 2048)
            self.attention1 = DeconvolutionModel1.AttentionBlock(2048, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.dropout1 = nn.Dropout(0.1)

        def forward(self, x):
            x = F.relu(self.fc1(x))  # First MLP layer
            x = self.attention1(x.unsqueeze(0)).squeeze(0)
            x = F.relu(self.fc2(x))  # Second MLP layer
            x = self.dropout1(x)
            return x

    class Decoder(nn.Module):
        def __init__(self, hidden_dim, output_dim, signature_matrix):
            super(DeconvolutionModel1.Decoder, self).__init__()
            self.fc2 = nn.Linear(512, 1024)
            self.fc3_fractions = nn.Linear(1024, output_dim)  # Output for cell fractions
            self.gep_matrix = nn.Parameter(torch.randn(input_dim, output_dim))
            self.signature_matrix = signature_matrix  # Store the signature matrix

        def forward(self, x):
            x = F.relu(self.fc2(x))
        
            # Predict cell fractions using min-max scaling
            cell_fractions = self.fc3_fractions(x)  # Get raw values
            min_vals, _ = cell_fractions.min(dim=-1, keepdim=True)  # Min values per row
            max_vals, _ = cell_fractions.max(dim=-1, keepdim=True)  # Max values per row

            # Min-max normalization to scale values between 0 and 1
            cell_fractions = (cell_fractions - min_vals) / (max_vals - min_vals + 1e-6)
            cell_fractions = cell_fractions / cell_fractions.sum(dim=-1, keepdim=True)  # Normalize rows to sum to 1

            # Reconstruct pseudobulk data (B = G * C)
            reconstructed_pseudobulk = torch.matmul(cell_fractions, self.signature_matrix.T)
        
            return cell_fractions, reconstructed_pseudobulk, self.gep_matrix

    def forward(self, x):
        encoded = self.encoder(x)  # Encode input
        cell_fractions, reconstructed_pseudobulk, gep = self.decoder(encoded)  # Decode to get predictions
        return cell_fractions, reconstructed_pseudobulk, gep

In [93]:
def compute_ccc(y_true, y_pred):
    """Compute Concordance Correlation Coefficient (CCC)"""
    mean_true = torch.mean(y_true)
    mean_pred = torch.mean(y_pred)
    covariance = torch.mean((y_true - mean_true) * (y_pred - mean_pred))
    variance_true = torch.var(y_true)
    variance_pred = torch.var(y_pred)

    ccc = (2 * covariance) / (variance_true + variance_pred + (mean_true - mean_pred) ** 2)
    return ccc.item()  # Return as a Python float

def compute_rmse(y_true, y_pred):
    """Compute Root Mean Square Error (RMSE)"""
    rmse = torch.sqrt(torch.mean((y_true - y_pred) ** 2))
    return rmse.item()  # Return as a Python float

In [96]:

start_time = time.time()

# Assuming DeconvolutionModel1 and DeconvolutionModel2 are defined as per the previous message
input_dim = X_train.shape[1]
output_dim = Y_train.shape[1] 
hidden_dim = 512
attention_dim = 2048
signature_dim = output_dim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize DeconvolutionModel1 and DeconvolutionModel2
model1 = DeconvolutionModel1(input_dim, hidden_dim, attention_dim, output_dim, signature_matrix_pbmc_expanded_tensor).to(device)


# Define loss function and optimizers for each model
loss_function = nn.MSELoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.0001)


l1 = 0.01 # Kl_div loss
l2 = 0.005 # pseudobulk_reconstruction_loss
l3 = 0.005  #pseudo loss1
l4 = 1  #sig_gep_loss


num_epochs = 100

# Define KL Divergence function
kl_loss_function = nn.KLDivLoss(reduction="batchmean")

# Training loop
for epoch in range(num_epochs):
    model1.train()


    # Forward pass through both models
    cell_fractions_pred, reconstructed1, gep_predictions1 = model1(X_train_tensor)

    # Loss calculations for model1
    train_loss1 = loss_function(cell_fractions_pred, Y_train_tensor)
    pseudobulk_reconstruction_loss1 = loss_function(X_train_tensor, reconstructed1)

    
    pseudo_bulk_pred_gep_and_Y_train1 = torch.matmul(Y_train_tensor, gep_predictions1.T)
    loss_pseudo_gep1 = loss_function(X_train_tensor, pseudo_bulk_pred_gep_and_Y_train1)

    loss_gep_sig1 = loss_function(gep_predictions1, signature_matrix_pbmc_expanded_tensor)

    # Define a prior distribution for the cell fractions (e.g., uniform prior)
    prior = torch.full_like(Y_train_tensor, fill_value=1.0 / output_dim).to(device)

    # Compute KL divergence loss for cell fractions
    kl_div_loss1 = kl_loss_function(torch.log_softmax(cell_fractions_pred, dim=-1), prior)

    loss_ccc = 1 - compute_ccc(Y_train_tensor, cell_fractions_pred)

    
    total_loss1 = train_loss1 + l1 * kl_div_loss1 + l2 * pseudobulk_reconstruction_loss1 + l3 * loss_pseudo_gep1 + l4 * loss_gep_sig1 


    # Backpropagation and optimizer step for both models
    optimizer1.zero_grad()
    total_loss1.backward()
    optimizer1.step()

    # Validation phase
    model1.eval()
   
    with torch.no_grad():
        val_cell_fractions1, val_reconstructed1, val_gep1 = model1(X_val_tensor)

        val_loss1 = loss_function(val_cell_fractions1, Y_val_tensor)

        val_ccc1 = compute_ccc(Y_val_tensor, val_cell_fractions1)

        val_rmse1 = compute_rmse(Y_val_tensor, val_cell_fractions1)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {total_loss1.item():.4f}, Val Loss: {val_loss1.item():.4f}, CCC: {val_ccc1:.4f}, RMSE: {val_rmse1:.4f}')

model1.eval()

with torch.no_grad():
    test_cell_fractions, test_pseudobulk_reconstructed, test_gep_predictions = model1(X_test_tensor)

print('CCC Value :', compute_ccc(Y_test_tensor, test_cell_fractions))

end_time = time.time()
print('Time taken :', end_time - start_time)

Epoch [10/100], Train Loss: 2.0018, Val Loss: 0.0017, CCC: 0.5666, RMSE: 0.0414
Epoch [20/100], Train Loss: 1.9984, Val Loss: 0.0008, CCC: 0.8157, RMSE: 0.0282
Epoch [30/100], Train Loss: 1.9958, Val Loss: 0.0005, CCC: 0.8773, RMSE: 0.0233
Epoch [40/100], Train Loss: 1.9934, Val Loss: 0.0004, CCC: 0.9107, RMSE: 0.0201
Epoch [50/100], Train Loss: 1.9910, Val Loss: 0.0003, CCC: 0.9251, RMSE: 0.0186
Epoch [60/100], Train Loss: 1.9887, Val Loss: 0.0003, CCC: 0.9325, RMSE: 0.0177
Epoch [70/100], Train Loss: 1.9864, Val Loss: 0.0003, CCC: 0.9349, RMSE: 0.0174
Epoch [80/100], Train Loss: 1.9842, Val Loss: 0.0003, CCC: 0.9378, RMSE: 0.0171
Epoch [90/100], Train Loss: 1.9819, Val Loss: 0.0003, CCC: 0.9385, RMSE: 0.0170
Epoch [100/100], Train Loss: 1.9796, Val Loss: 0.0003, CCC: 0.9389, RMSE: 0.0170
CCC Value : 0.9523104429244995
Time taken : 5.818882942199707


In [97]:
# Using predicted cell fractions from the test data set to get the GEPs on the test data set
start_time = time.time()

input_dim = X_train.shape[1]
output_dim = Y_train.shape[1] 
hidden_dim = 256
attention_dim = 2048
signature_dim = output_dim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize DeconvolutionModel1 and DeconvolutionModel2
model2 = DeconvolutionModel1(input_dim, hidden_dim, attention_dim, output_dim, signature_matrix_pbmc_expanded_tensor).to(device)


# Initialize the ensemble model and evaluation setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizers for each model
loss_function = nn.MSELoss()
optimizer1 = optim.Adam(model2.parameters(), lr=0.0001)



l1 = 0.01 # Kl_div loss
l2 = 0.01 # pseudobulk_reconstruction_loss
l3 = 0.01  #pseudo loss1
l4 = 1
l5 = 0.00


num_epochs = 100

# Define KL Divergence function
kl_loss_function = nn.KLDivLoss(reduction="batchmean")

# Training loop
for epoch in range(num_epochs):
    model2.train()


    # Forward pass through both models
    cell_fractions_pred, reconstructed1, gep_predictions1 = model2(X_test_tensor)

    # Loss calculations for model1
    train_loss1 = loss_function(cell_fractions_pred, test_cell_fractions)
    pseudobulk_reconstruction_loss1 = loss_function(X_test_tensor, reconstructed1)

    
    pseudo_bulk_pred_gep_and_Y_train1 = torch.matmul(test_cell_fractions, gep_predictions1.T)
    loss_pseudo_gep1 = loss_function(X_test_tensor, pseudo_bulk_pred_gep_and_Y_train1)

    loss_gep_sig1 = loss_function(gep_predictions1, signature_matrix_pbmc_expanded_tensor)

    # Define a prior distribution for the cell fractions (e.g., uniform prior)
    prior = torch.full_like(test_cell_fractions, fill_value=1.0 / output_dim).to(device)

    # Compute KL divergence loss for cell fractions
    kl_div_loss1 = kl_loss_function(torch.log_softmax(cell_fractions_pred, dim=-1), prior)

    loss_ccc = 1 - compute_ccc(test_cell_fractions, cell_fractions_pred)

    
    total_loss1 = train_loss1 + l1 * kl_div_loss1 + l2 * pseudobulk_reconstruction_loss1 + l3 * loss_pseudo_gep1 + l4 * loss_gep_sig1 


    # Backpropagation and optimizer step for both models
    optimizer1.zero_grad()
    total_loss1.backward()
    optimizer1.step()


    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {total_loss1.item():.4f}')

end_time = time.time()
print('Time taken for GEP prediction:', end_time - start_time)

Epoch [10/100], Train Loss: 2.0265
Epoch [20/100], Train Loss: 2.0236
Epoch [30/100], Train Loss: 2.0212
Epoch [40/100], Train Loss: 2.0188
Epoch [50/100], Train Loss: 2.0165
Epoch [60/100], Train Loss: 2.0142
Epoch [70/100], Train Loss: 2.0119
Epoch [80/100], Train Loss: 2.0096
Epoch [90/100], Train Loss: 2.0074
Epoch [100/100], Train Loss: 2.0051
Time taken for GEP prediction: 3.8916890621185303


In [98]:
gep_predictions1

Parameter containing:
tensor([[-1.1264,  1.3519,  1.1864,  ..., -1.0992, -0.0513, -0.0311],
        [-0.8558, -0.8413,  0.3248,  ..., -0.3009, -0.2373, -0.6434],
        [-0.6709,  1.4279,  1.3562,  ..., -0.7709, -0.3250,  1.1454],
        ...,
        [-0.1552,  0.0960, -0.7116,  ..., -0.9668,  1.4226, -0.3888],
        [ 0.3426,  0.1337,  0.7645,  ...,  1.4417, -1.2608,  0.8624],
        [ 0.5247, -0.0580,  0.4545,  ...,  1.1279, -0.0758, -1.2813]],
       device='cuda:0', requires_grad=True)